In [2]:
import pandas as pd
import numpy as np
from utils import *

In [7]:
train_llm=pd.read_csv('../dataset/llm_annotations/train_llm_ann.csv')
test_llm=pd.read_csv('../dataset/llm_annotations/test_llm_ann.csv')
val_llm=pd.read_csv('../dataset/llm_annotations/val_llm_ann.csv')


In [29]:
from collections import Counter

def create_majority_label_instr(df):
    def get_majority(row):
        labels = [row['olmo_instruct'], row['mistral_instruct'], row['llama_instruct']]
        label_counts = Counter(labels)
        
        # Trova la label più comune e il suo conteggio
        most_common = label_counts.most_common(1)[0]
        
        if most_common[1] > 1:
            # Se la label più comune appare più di una volta, è la maggioranza
            return most_common[0]
        else:
            # Se non c'è una maggioranza chiara
            return "No majority"
    
    df['majority_label'] = df.apply(get_majority, axis=1)
    return df 

In [ ]:
from collections import Counter

def create_majority_label(df):
    def get_majority(row):
        labels = [row['olmo'], row['mistral'], row['llama']]
        label_counts = Counter(labels)
        
        # Trova la label più comune e il suo conteggio
        most_common = label_counts.most_common(1)[0]
        
        if most_common[1] > 1:
            # Se la label più comune appare più di una volta, è la maggioranza
            return most_common[0]
        else:
            # Se non c'è una maggioranza chiara
            return "No majority"
    
    df['majority_label'] = df.apply(get_majority, axis=1)
    return df

In [30]:
train_llm_i=create_majority_label_instr(train_llm)
test_llm_i=create_majority_label_instr(test_llm)
val_llm_i=create_majority_label_instr(val_llm)

In [13]:
train_llm = create_majority_label(train_llm)
test_llm = create_majority_label(test_llm)
val_llm = create_majority_label(val_llm)

In [15]:
count_no_majority_train_llm = train_llm['majority_label'].value_counts().get("No majority", 0)
count_no_majority_test_llm = test_llm['majority_label'].value_counts().get("No majority", 0)
count_no_majority_val_llm = val_llm['majority_label'].value_counts().get("No majority", 0)

In [32]:
count_no_majority_train_llm_i = train_llm_i['majority_label'].value_counts().get("No majority", 0)
count_no_majority_test_llm_i = test_llm_i['majority_label'].value_counts().get("No majority", 0)
count_no_majority_val_llm_i = val_llm_i['majority_label'].value_counts().get("No majority", 0)

In [33]:
count_no_majority_train_llm_i,count_no_majority_test_llm_i,count_no_majority_val_llm_i

(284, 73, 61)

In [16]:
count_no_majority_train_llm,count_no_majority_test_llm, count_no_majority_val_llm

(195, 46, 54)

In [8]:
train = pd.read_csv('../dataset/train_gpt.csv')
test = pd.read_csv('../dataset/test_gpt.csv')
val = pd.read_csv('../dataset/val_gpt.csv')

In [9]:
def clean_data_new(df):
    
    df = df[df['majority_label'] != 'link broken']
    return df

def verify_filtering_new(df):
    print(f"Numero di righe: {df.shape[0]}")
    print(f"Numero di colonne: {df.shape[1]}")

In [18]:
train_llm= clean_data_new(train_llm)
verify_filtering_new(train_llm)

test_llm = clean_data_new(test_llm)
verify_filtering_new(test_llm)

val_llm = clean_data_new(val_llm)
verify_filtering_new(val_llm)

Numero di righe: 619
Numero di colonne: 14
Numero di righe: 139
Numero di colonne: 14
Numero di righe: 139
Numero di colonne: 14


In [34]:
train_llm_i= clean_data_new(train_llm_i)
verify_filtering_new(train_llm_i)

test_llm_i = clean_data_new(test_llm_i)
verify_filtering_new(test_llm_i)

val_llm_i = clean_data_new(val_llm_i)
verify_filtering_new(val_llm_i)

Numero di righe: 619
Numero di colonne: 14
Numero di righe: 139
Numero di colonne: 14
Numero di righe: 139
Numero di colonne: 14


In [10]:
train= clean_data_new(train)
verify_filtering_new(train)

test = clean_data_new(test)
verify_filtering_new(test)

val = clean_data_new(val)
verify_filtering_new(val)

Numero di righe: 734
Numero di colonne: 19
Numero di righe: 157
Numero di colonne: 19
Numero di righe: 157
Numero di colonne: 19


In [11]:
count_no_majority_train = train['majority_label'].value_counts().get("No majority", 0)
count_no_majority_test = test['majority_label'].value_counts().get("No majority", 0)
count_no_majority_val = val['majority_label'].value_counts().get("No majority", 0)

In [12]:
count_no_majority_train,count_no_majority_test,count_no_majority_val

(101, 18, 15)

In [13]:
train['gpt_summaries'] = train['gpt_summaries'].fillna('')
test['gpt_summaries'] = test['gpt_summaries'].fillna('')
val['gpt_summaries'] = val['gpt_summaries'].fillna('')

In [14]:
print(train.shape)
print(test.shape)
print(val.shape)

(734, 19)
(157, 19)
(157, 19)


In [15]:
def doc_cocat(row):
    if row['gpt_summaries'] != '':
        return row['gpt_summaries']
    else:
        return row['docCont']

train['doc'] = train.apply(doc_cocat, axis=1)
test['doc'] = test.apply(doc_cocat, axis=1)
val['doc'] = val.apply(doc_cocat, axis=1) 

In [16]:
def combine_columns(df): #combine to create Input
    df['Input'] = df['Query'] + ' ' + df['docTitle'] + '. ' + df['doc']
    return df

train = combine_columns(train)
test = combine_columns(test)
val = combine_columns(val)

In [17]:
train['doc'] = train['doc'].str.replace('\n', ' ')
test['doc'] = test['doc'].str.replace('\n', ' ')
val['doc'] = val['doc'].str.replace('\n', ' ')

train['Input'] = train['Input'].str.replace('\n', ' ')
test['Input'] = test['Input'].str.replace('\n', ' ')
val['Input'] = val['Input'].str.replace('\n', ' ')

In [18]:
train['majority_label'].value_counts() 

majority_label
Neutral        203
Pro            158
Against        142
Not-about      130
No majority    101
Name: count, dtype: int64

In [75]:
import os


def create_directory(directory_path):
    """
    Crea una nuova directory se non esiste già.
    """
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"Directory '{directory_path}' creata.")
    else:
        print(f"Directory '{directory_path}' già esistente.")

def save_dataset(train, test, val, output_directory, prefix):
    """
    Salva i DataFrame come file CSV nella directory di output con i nomi prefissati.
    """
    # Assicurati che la directory esista
    create_directory(output_directory)
    
    # Costruisci i percorsi completi per i file di output
    train_path = os.path.join(output_directory, f'{prefix}_train.csv')
    test_path = os.path.join(output_directory, f'{prefix}_test.csv')
    val_path = os.path.join(output_directory, f'{prefix}_val.csv')
    
    # Salva i DataFrame come file CSV
    train.to_csv(train_path, index=False)
    test.to_csv(test_path, index=False)
    val.to_csv(val_path, index=False)

    print(f"Dataset salvati nella directory '{output_directory}'.")


# Specifica il percorso della nuova directory e i nomi dei file
output_directory = '../dataset/multip_new'
prefix = 'multip'

# Usa la funzione per salvare i dataset nella nuova directory
save_dataset(train, test, val, output_directory, prefix)


Directory '../dataset/multip_new' creata.
Dataset salvati nella directory '../dataset/multip_new'.


In [72]:
output_directory = '../dataset/multip' #with no majority instances 


save_dataset(train, test, val, output_directory, 'multip') 

Train DataFrame shape: (2199, 7)
Test DataFrame shape: (471, 7)
Val DataFrame shape: (471, 7)
Train, Test, and Val datasets for 'multip' saved to ../dataset/multip


In [68]:
train['majority_label'].value_counts() 

majority_label
Neutral        203
Pro            158
Against        142
Not-about      130
No majority    101
Name: count, dtype: int64

In [25]:
col_name=["answer1","answer2","answer3","majority_label"]
condition = train[col_name].apply(lambda col: col.str.contains('Link-broken', na=False)).any(axis=1)

# Filtra le righe che soddisfano la condizione
link_broken_rows = train[condition]

In [26]:
link_broken_rows

,QID_x,Topic,Query,docID,docURL,docTitle,docCont,engineID,rank,answer1,answer2,answer3,If there is a link-broken option,majority_label,labels,Input,Input_length,docCont_length,gpt_summaries,doc
17,2,Euthanasia & Assisted Suicide,Should Euthanasia or Physician-Assisted Suicid...,1s2q1r,http://theconversation.com/viewpoints-should-e...,Viewpoints: should euthanasia be available for...,"Research Associate, University of Notre Dame A...",1,1,Against,Link-broken,Neutral,1,No majority,"['Against', 'Link-broken', 'Neutral']",Should Euthanasia or Physician-Assisted Suicid...,1597,1580,The debate surrounding the legalization of eut...,The debate surrounding the legalization of eut...
41,37,Lowering the Voting Age to 16,Should the Voting Age Be Lowered to 16?,1s37q6r,https://thehill.com/homenews/state-watch/43067...,Oregon lawmakers seek to lower state voting ag...,Democratic lawmakers in Oregon announced a mea...,1,6,Link-broken,Pro,Pro,1,Pro,"['Link-broken', 'Pro', 'Pro']",Should the Voting Age Be Lowered to 16? Oregon...,290,270,,Democratic lawmakers in Oregon announced a mea...
87,9,Obamacare,Is the Patient Protection and Affordable Care ...,2s9q10r,http://newsroom.ucla.edu/in-the-news/ucla-in-t...,"UCLA In the News February 20, 2019 | UCLA",UCLA In the News lists selected mentions of UC...,2,10,Link-broken,Not-about,Not-about,1,Not-about,"['Link-broken', 'Not-about', 'Not-about']",Is the Patient Protection and Affordable Care ...,1480,1459,The provided text from UCLA News dated Februar...,The provided text from UCLA News dated Februar...
100,14,Tablets vs. Textbooks,Should Tablets Replace Textbooks in K-12 Schools?,1s14q8r,http://www.itp.net/602157-the-smart-campus-net...,The smart campus network - ITP.net,Many education institutions are beginning to m...,1,8,Pro,Link-broken,Pro,1,Pro,"['Pro', 'Link-broken', 'Pro']",Should Tablets Replace Textbooks in K-12 Schoo...,959,946,The educational landscape is increasingly embr...,The educational landscape is increasingly embr...
119,20,Banned Books,Should Parents or Other Adults Be Able to Ban ...,2s20q7r,https://www.news-leader.com/story/news/educati...,Want to ban books? Springfield has long histor...,Have an existing account?Already have a subscr...,2,7,Neutral,Neutral,Link-broken,1,Neutral,"['Neutral', 'Neutral', 'Link-broken']",Should Parents or Other Adults Be Able to Ban ...,93,68,,Have an existing account?Already have a subscr...
137,1,Medical Marijuana,Should Marijuana Be a Medical Option?,1s1q10r,https://patch.com/massachusetts/brookline/anno...,Healthier Ways to Use Medical Marijuana Beside...,Medical cannabis (medical marijuana) refers to...,1,10,Link-broken,Pro,Neutral,1,No majority,"['Link-broken', 'Pro', 'Neutral']",Should Marijuana Be a Medical Option? Healthie...,1034,1016,"Medical marijuana, also known as medical canna...","Medical marijuana, also known as medical canna..."
165,42,Prostitution - Legalize It,Should Prostitution Be Legal?,2s42q8r,https://www.washingtonpost.com/national/a-neva...,A Nevada county takes aim at legal prostitutio...,Desktop notifications are on | Turn off Get ...,2,8,Link-broken,Neutral,Neutral,1,Neutral,"['Link-broken', 'Neutral', 'Neutral']",Should Prostitution Be Legal? A Nevada county ...,1322,1300,"Lyon County, Nevada, is facing a critical deci...","Lyon County, Nevada, is facing a critical deci..."
190,2,Euthanasia & Assisted Suicide,Should Euthanasia or Physician-Assisted Suicid...,1s2q5r,https://www.medscape.com/viewarticle/909233,Criminal Case Spurs a Rethink of Euthanasia fo...,The recent launch of a criminal investigation ...,1,5,Neutral,Link-broken,Against,1,No majority,"['Neutral', 'Link-broken', 'Against']",Should Euthanasia or Physician-Assisted Suicid...,79,62,,The recent launch of a criminal investigation ...
257,57,Penny - Keep It,Should the Penny Stay in Circulation?,2s57q8r,https://www.govexec.com/management/2018/07/us-...,The U.S. Mint Lost $69 Million Making Pennies ...,The cooper and zinc that go into making small ...,2,8,

In [19]:
import os
directory = '/home/benedetta.muscato/multiperspective/dataset/multip_new'
train_file = os.path.join(directory, 'train_multip_human.csv')
test_file = os.path.join(directory, 'test_multip_human.csv')
val_file = os.path.join(directory, 'val_multip_human.csv')


train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)
val.to_csv(val_file, index=False) 

In [3]:
train_df=pd.read_csv("/home/benedetta.muscato/multiperspective/dataset/multip_new/multip_train.csv")
val_df=pd.read_csv("/home/benedetta.muscato/multiperspective/dataset/multip_new/multip_val.csv")
test_df=pd.read_csv("/home/benedetta.muscato/multiperspective/dataset/multip_new/multip_test.csv")

In [6]:
cols_of_interest = ['answer1', 'answer2', 'answer3']
rows_with_nan = train_df[train_df[cols_of_interest].isna().any(axis=1)]
print(rows_with_nan)

     QID_x                  Topic  \
159     11  Prescription Drug Ads   
274     11  Prescription Drug Ads   
330     11  Prescription Drug Ads   

                                                 Query    docID  \
159  Should Prescription Drugs Be Advertised Direct...  1s11q5r   
274  Should Prescription Drugs Be Advertised Direct...  1s11q1r   
330  Should Prescription Drugs Be Advertised Direct...  1s11q2r   

                                                docURL  \
159  http://www.newstalkzb.co.nz/on-air/the-weekend...   
274  http://www.teenink.com/hot_topics/health/artic...   
330  https://www.phillyvoice.com/why-us-has-higher-...   

                                              docTitle  \
159  Why drug advertising is a 'costly exercise' th...   
274  Should Prescription Drugs Be Directly Advertis...   
330  Why the US has higher drug prices than other c...   

                                               docCont  engineID  rank  \
159  This is a modal window.Beginning of d

In [4]:
test_df['majority_label'].value_counts()

majority_label
Pro            43
Neutral        43
Against        29
Not-about      24
No majority    18
Name: count, dtype: int64

In [20]:
test_llm

,docID,Query,docTitle,doc,Input,labels,majority_label,Prompt,olmo,olmo_instruct,llama,llama_instruct,mistral_instruct,mistral
0,1s54q9r,Should Social Security Be Privatized?,Social Security: Why America Can and Should Al...,The debate surrounding the privatization of So...,Should Social Security Be Privatized? Social S...,"['Pro', 'Pro', 'Pro']",No majority,You're an annotator chosen for a task of annot...,Pro,Against,Neutral,Neutral,Pro,Against
1,2s28q5r,Can Alternative Energy Effectively Replace Fos...,Trump’s speech ignored global warming and clim...,"In 2018, the U.S. witnessed significant weathe...",Can Alternative Energy Effectively Replace Fos...,"['Pro', 'Pro', 'Pro']",No majority,You're an annotator chosen for a task of annot...,Not-about,Pro,Against,Neutral,Not-about,Pro
2,2s39q1r,Should the United States Maintain Its Embargo ...,Cuba: Trouble in Paradise | Havana Times,The article examines the economic status and c...,Should the United States Maintain Its Embargo ...,"['Against', 'Neutral', 'Against']",Pro,You're an annotator chosen for a task of annot...,Not-about,Against,Pro,Neutral,Pro,Pro
3,1s52q3r,Should the United States Return to a Gold Stan...,Maduro calls for return of Venezuela's UK-depo...,Caracas (AFP) - Venezuela President Nicolas Ma...,Should the United States Return to a Gold Stan...,"['Not-about', 'Not-about', 'Not-about']",Neutral,You're an annotator chosen for a task of annot...,Neutral,Pro,Neutral,Neutral,Not-about,Pro
4,2s8q3r,Is Obesity a Disease?,Treatment for obesity and fatty liver disease ...,Professor Amiram Goldblum and his team at the ...,Is Obesity a Disease? Treatment for obesity an...,"['Pro', 'Not-about', 'Not-about']",Pro,You're an annotator chosen for a task of annot...,Not-about,Not-about,Pro,Neutral,Pro,Pro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,1s48q7r,Are the Olympic Games an Overall Benefit for T...,Business of the Olympics - InvestorGuide.com,"With this year’s Rio Olympics beginning, the b...",Are the Olympic Games an Overall Benefit for T...,"['Neutral', 'Pro', 'Pro']",Pro,You're an annotator chosen for a task of annot...,Against,Against,Pro,Against,Pro,Pro
135,2s21q4r,Should the Drinking Age Be Lowered from 21 to ...,Lena Dunham's Comments Defending Jordyn Woods ...,Amid all the cheating rumors surrounding Jordy...,Should the Drinking Age Be Lowered from 21 to ...,"['Not-about', 'Not-about', 'Not-about']",No majority,You're an annotator chosen for a task of annot...,Against,Not-about,Neutral,Neutral,Not-about,Pro
136,2s7q8r,Should People Become Vegetarian?,Vegetarian man devastated after accidentally e...,Pizza Hut has apologisedA man who has never ea...,Should People Become Vegetarian? Vegetarian ma...,"['Neutral', 'Neutral', 'Not-about']",Pro,You're an annotator chosen for a task of annot...,Pro,Not-about,Pro,Not-about,Not-about,Not-about
137,2s20q5r,Should Parents or Other Adults Be Able to Ban ...,Community Montessori’s 100th day of school: Ki...,Hi Already a subscriber?Subscribe today for fu...,Should Parents or Other Adults Be Able to Ban ...,"['Not-about', 'Not-about', 'Neutral']",No majority,You're an annotator chosen for a task of annot...,Against,Not-about,Not-about,Pro,Not-about,Pro


In [27]:

import os
directory = '/home/benedetta.muscato/multiperspective/dataset/multip_new'
train_file = os.path.join(directory, 'train_llm.csv')
test_file = os.path.join(directory, 'test_llm.csv')
val_file = os.path.join(directory, 'val_llm.csv')


train_llm.to_csv(train_file, index=False)
test_llm.to_csv(test_file, index=False)
val_llm.to_csv(val_file, index=False) 

In [ ]:
import os
directory = '/home/benedetta.muscato/multiperspective/dataset/multip_new'
train_file = os.path.join(directory, 'train_llm_instr.csv')
test_file = os.path.join(directory, 'test_llm_instr.csv')
val_file = os.path.join(directory, 'val_llm_instr.csv')


train_llm_i.to_csv(train_file, index=False)
test_llm_i.to_csv(test_file, index=False)
val_llm_i.to_csv(val_file, index=False) 